In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obesity"
cohort = "GSE281144"

# Input paths
in_trait_dir = "../../input/GEO/Obesity"
in_cohort_dir = "../../input/GEO/Obesity/GSE281144"

# Output paths
out_data_file = "../../output/preprocess/Obesity/GSE281144.csv"
out_gene_data_file = "../../output/preprocess/Obesity/gene_data/GSE281144.csv"
out_clinical_data_file = "../../output/preprocess/Obesity/clinical_data/GSE281144.csv"
json_path = "../../output/preprocess/Obesity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gut Adaptation After Gastric Bypass in Humans Reveals Metabolically Significant Shift in Fuel Metabolism"
!Series_summary	"Objective: Roux-en-Y gastric bypass surgery (RYGB) is among the most effective therapies for obesity and type 2 diabetes (T2D), and intestinal adaptation is a proposed mechanism for these effects. We hypothesized that intestinal adaptation precedes and relates to metabolic improvement in humans after RYGB."
!Series_summary	"Methods: This was a prospective, longitudinal first-in-human study of gene expression (GE) in Roux limb (RL) collected surgically/endoscopically from 19 patients with and without diabetes. GE was determined by microarray across 6 postoperative months, including at an early postoperative (1 month  15 days) timepoint."
!Series_summary	"Results: RL GE demonstrated tissue remodeling and metabolic reprogramming, including increased glucose and amino acid utilization. RL GE signatures were established early, bef

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analysis of data availability and preparation for feature extraction
import pandas as pd

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression (GE) data
# from microarray analysis, which is suitable for our study.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Obesity): 
# The dataset is about gastric bypass in patients with obesity (with or without diabetes).
# The trait is constant (all subjects have obesity) according to the background information.
# Therefore, we cannot use this dataset for association studies of obesity vs. non-obesity.
trait_row = None  # Obesity status is not available as a variable (all subjects have obesity)

# Diabetes status is available in row 1
# This could be considered as an alternative trait since it varies among subjects
diabetes_row = 1

# Age:
# There's no information about age in the sample characteristics
age_row = None

# Gender:
# Gender information is available in row 0
gender_row = 0

# 2.2 Data Type Conversion

# Since we can't use obesity as our trait variable (all subjects have it),
# we'll use diabetes status instead, which is binary
def convert_trait(value):
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
        if value.lower() == "diabetic":
            return 1
        elif value.lower() == "control":
            return 0
    return None

# Age conversion function (not used in this dataset)
def convert_age(value):
    return None

# Gender conversion function
def convert_gender(value):
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
        if value.lower() == "female":
            return 0
        elif value.lower() == "male":
            return 1
    return None

# 3. Save Metadata
# Since trait_row is None (all subjects have obesity), we'll set is_trait_available to False
is_trait_available = False

# Validate and save cohort info
initial_validation = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this step


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (e.g., "2824546_st") appear to be Affymetrix probe IDs 
# rather than standard human gene symbols like BRCA1, TP53, etc.
# Affymetrix probe IDs typically need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# From examining the gene expression data and annotation data:
# 1. The gene expression data's index contains probe IDs like '2824546_st'
# 2. The gene annotation data has columns:
#    - 'ID': These appear to be transcript IDs like 'TC01000001.hg.1'
#    - 'probeset_id': Same as ID
#    - 'gene_assignment': Contains gene symbols with additional information

# The identifiers in gene expression data don't directly match the 'ID' column in annotation
# This is a common issue with Affymetrix arrays, where the actual probe IDs in the expression matrix
# may not match exactly with the annotation file IDs

# First, let's see how the IDs from gene expression data map to the annotation file
print("Expression data index example:", gene_data.index[0])
print("Number of probes in expression data:", len(gene_data.index))
print("Number of probes in annotation data:", len(gene_annotation))

# We need to establish a mapping between probe IDs and gene symbols
# The 'gene_assignment' column contains gene symbols

# Let's examine if there's a way to extract just the gene symbols from the gene_assignment column
# Extract gene mapping using ID column and gene_assignment column
gene_mapping = gene_annotation[['ID', 'gene_assignment']].copy()
gene_mapping = gene_mapping.rename(columns={'gene_assignment': 'Gene'})

# The 'Gene' column contains complex strings with gene symbols embedded
# We'll use the extract_human_gene_symbols function to extract gene symbols
print("First few rows of mapping before extraction:")
print(gene_mapping.head(2))

# Apply the gene mapping
gene_data_mapped = apply_gene_mapping(gene_data, gene_mapping)

# Normalize gene symbols in the index to handle cases where different probe IDs map to the same gene
gene_data = normalize_gene_symbols_in_index(gene_data_mapped)

# Print statistics about the resulting gene expression data
print("\nGene expression data after mapping:")
print(f"Number of genes: {len(gene_data.index)}")
print(f"Number of samples: {gene_data.shape[1]}")
print("First few genes:")
print(gene_data.index[:10])


Expression data index example: 2824546_st
Number of probes in expression data: 70523
Number of probes in annotation data: 2468569
First few rows of mapping before extraction:
                ID                                               Gene
0  TC01000001.hg.1  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  TC01000002.hg.1  ENST00000408384 // MIR1302-11 // microRNA 1302...



Gene expression data after mapping:
Number of genes: 24018
Number of samples: 34
First few genes:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1',
       'A2ML1-AS2', 'A2MP1', 'A4GALT'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since trait_row is None (as determined in Step 2), we skip the clinical feature extraction
# and data linking steps as instructed.
print("No trait data available for this dataset (all subjects have obesity). Skipping clinical feature extraction.")

# Create a linked_data DataFrame with only gene expression data for the final validation
# This matches the expected input format for validate_and_save_cohort_info
linked_data = normalized_gene_data.copy()

# Final validation - mark the dataset as not usable for trait studies
note = "All subjects in this dataset have obesity, making it unsuitable for obesity vs. non-obesity association studies."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,  # False as determined in Step 2
    is_biased=True,  # No variability in the trait (all have obesity)
    df=linked_data,
    note=note
)

print(f"Dataset usability status: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for obesity trait-gene association studies.")

Normalized gene data saved to ../../output/preprocess/Obesity/gene_data/GSE281144.csv
No trait data available for this dataset (all subjects have obesity). Skipping clinical feature extraction.
Dataset usability status: False
Dataset is not usable for obesity trait-gene association studies.
